In [1]:
%cd src

/home/ubuntu/rebel/src


/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from hydra import compose, initialize
import pytorch_lightning as pl
from pl_data_modules import BasePLDataModule
from pl_modules import BasePLModule
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer
from pytorch_lightning.core.saving import _load_from_checkpoint
from typing import cast
import torch
from pytorch_lightning.utilities.migration.utils import _pl_migrate_checkpoint
from pathlib import Path
from pytorch_lightning.core.saving import _load_state

checkpoint_path = "/home/ubuntu/rebel/outputs/2024-08-21/17-25-14/experiments/default_name/last.ckpt"
seed = 42
config_name = 'google/mt5-base'
tokenizer_name = 'google/mt5-base'

/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
initialize(config_path="./conf")
conf = compose(config_name="root", overrides=[f"checkpoint_path={checkpoint_path}", "do_predict=True", "do_eval=True"])

/tmp/ipykernel_685160/1858989061.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="./conf")


In [4]:
pl.seed_everything(conf.seed)

config = AutoConfig.from_pretrained(
    conf.config_name if conf.config_name else conf.model_name_or_path,
    decoder_start_token_id = 0,
    early_stopping = False,
    no_repeat_ngram_size = 0,
)

special_tokens = [
    "<triplet>",
    "<obj>",
    "<subj>",
]
    
tokenizer_kwargs = {
    "use_fast": True,  # Always use fast tokenizer for better compatibility
    "additional_special_tokens": special_tokens, 
    "legacy": False,  # For mt5
}

tokenizer = AutoTokenizer.from_pretrained(
    conf.tokenizer_name if conf.tokenizer_name else conf.model_name_or_path,
    **tokenizer_kwargs
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    conf.model_name_or_path,
    config=config,
)
# if not conf.finetune:
model.resize_token_embeddings(len(tokenizer))

Seed set to 42
/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Embedding(250103, 768)

In [5]:
# data module declaration
pl_data_module = BasePLDataModule(conf, tokenizer, model)

# main module declaration
pl_module = BasePLModule(conf, config, tokenizer, model)
device = torch.device("cuda:0")

In [6]:
checkpoint = torch.load(
    conf.checkpoint_path,
    map_location=torch.device('cuda'),
)
checkpoint = _pl_migrate_checkpoint(
    checkpoint, checkpoint_path=(conf.checkpoint_path if isinstance(conf.checkpoint_path, (str, Path)) else None)
)
pl_module = _load_state(BasePLModule, checkpoint, config=config, tokenizer=tokenizer, model=model)
state_dict = checkpoint["state_dict"]
pl_module = cast(BasePLModule, pl_module.to(device))

/tmp/ipykernel_685160/549381607.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(


In [7]:
# pl_module.hparams.predict_with_generate = True
pl_module.hparams.test_file = pl_data_module.conf.test_file
# trainer
trainer = pl.Trainer(
        num_nodes=conf.gpus,
)
# Manually run prep methods on DataModule
pl_data_module.prepare_data()
pl_data_module.setup('test')


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Map:   0%|                                                                         | 0/517701 [00:00<?, ? examples/s]/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize 

In [9]:
result = trainer.test(pl_module, dataloaders=pl_data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(

Testing DataLoader 0:  16%|██████████▍                                                      | 287/1794 [12:50<1:07:24,  0.37it/s]
Testing: |                                                                                    | 1794/? [1:19:15<00:00,  0.38it/s]RE Evaluation in *** BOUNDARIES *** mode
processed 33281 sentences with 90588 relations; found: 73833 relations; correct: 56530.
	ALL	 TP: 56530;	FP: 17096;	FN: 33539
		(m avg): precision: 76.78;	recall: 62.76;	f1: 69.07 (micro)
		(M avg): precision: 24.23;	recall: 12.44;	f1: 14.84 (Macro)

	relation: 	TP: 0;	FP: 0;	FN: 0;	precision: 0.00;	recall: 0.00;	f1: 0.00;	0
	کشور: 	TP: 20931;	FP: 3060;	FN: 3128;	precision: 87.25;	recall: 87.00;	f1: 87.12;	23991
	موقعیت در تقسیمات کشوری: 	TP: 9122;	FP: 1899;	FN: 3769;	precision: 82.77;	recall: 70.76;	f1: 76.30;	11021
	عضو تیم ورزشی: 	TP: 96;	FP: 26;	FN: 80;	precision: 78.69;	recall: 54.55;	f1: 64.43;	122
	زیربخش: 	TP: 8269;	FP: 1811;	FN: 1668;	precision: 82.03;	recall: 83.21;	f1: 82.62;	1008

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_F1_micro       │     69.06747436523438     │
│         test_loss         │    0.15421459078788757    │
│      test_prec_micro      │     76.77994537353516     │
│     test_recall_micro     │     62.76299285888672     │
└───────────────────────────┴───────────────────────────┘

In [12]:
result

[{'test_loss': 0.15421459078788757,
  'test_prec_micro': 76.77994537353516,
  'test_recall_micro': 62.76299285888672,
  'test_F1_micro': 69.06747436523438}]

In [15]:
from transformers import default_data_collator
from torch.utils.data import DataLoader

def inference_dataloader(dataset, batch_size=8, label_pad_token_id=-100, drop_last=False, num_workers=4, pin_memory=True) -> DataLoader:
    return DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=default_data_collator,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False
    )


In [18]:
examples = pl_data_module.test_dataset[0]

In [19]:
examples

{'input_ids': [10126,
  74494,
  42591,
  274,
  7298,
  3814,
  2580,
  259,
  271,
  2665,
  10882,
  10948,
  33719,
  29781,
  1164,
  376,
  5320,
  3816,
  950,
  934,
  10882,
  406,
  695,
  14085,
  376,
  5215,
  46155,
  5677,
  950,
  259,
  260,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [250100,
  10126,
  74494,
  42591,
  134030,
  3814,
  2580,
  271,
  250102,
  1164,
  376,
  5320,
  3816,
  250101,
  6034,
  1]}

In [ ]:
dataset = [
    {
        ""
    }
]